In [ ]:
# Your ID : sign in https://gportal.jaxa.jp/gpr/?lang=ja

# Product Name : You can check from https://gportal.jaxa.jp/gpr/assets/mng_upload/GCOM-C/GCOM-C_SHIKISAI_Data_Users_Handbook_jp.pdf
#                page 4-20
        
# Version : '1','2','3'
#             Different versions provide data for different periods
#             check detail from https://shikisai.jaxa.jp/repro_stat_j.html
                
# Date : e.g. '20220322'

# Temporal : daily：'01D' , 8-days ：'08D' , Monthly : '01M'

# Obrit : Ascending：'A' , Descending：'D'
    
# Target_path : e.g. 'xxx/SGLI/'


        

In [24]:
from ftplib import FTP
class SGLI_L2_Downloader:
    
    def __init__ (self, User_ID , ProductName, Ver, Date , Tile , Obrit ,ParaVer, Target_path):
        self.User_ID = User_ID
        self.ProductName = ProductName
        self.Ver = Ver
        self.Date = Date
        self.Tile = Tile
        self.Obrit = Obrit
        self.ParaVer = ParaVer
        self.Target_path = Target_path
        
        
    def path (self):
        return '/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.{}/{}/{}/{}/{}'.format(self.ProductName, self.Ver, self.Date[0:4], self.Date[4:6], self.Date[6:8])

    def filename (self):
        return 'GC1SG1_{}{}01D_T{}_L2SG_{}Q_{}00{}.h5'.format(self.Date, self.Obrit, self.Tile, self.ProductName, self.Ver, self.ParaVer)
    
    def login (self):
        ftp_addr = 'ftp.gportal.jaxa.jp'
        f = FTP(ftp_addr)
        f.login(self.User_ID,'anonymous')
        
        return f
    
    def download (self, f):
        remote_filepath = self.path()
        f.cwd(remote_filepath)
        list=f.nlst()

        bufsize=1024
        data = open ('{}{}'.format(self.Target_path, self.filename()), 'wb')
        filename = 'RETR ' + self.filename()
        f.retrbinary(filename,data.write,bufsize)
        
    def logout (self, f):
        f.quit()
    
def reporjection_Geo(filename,Band,Target_path):
    
    #print(path)
    os.system('/media/liwei/Data/SGLI/SGLI_geo_map_linux.exe {} -d Geometry_data/{} -o {}'.format(filename,Band,Target_path[:-1]))
    
def reporjection_Image(filename,Band,Target_path):
    path = Target_path + filename
    #print(path)
    os.system('/media/liwei/Data/SGLI/SGLI_geo_map_linux.exe {} -d Image_data/{} -o {}'.format(filename,Band,Target_path[:-1])) 
        
        
        
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        


def SGLI_2_AHI_IMAGE(filepath,output_path):
    geotiff_da = xr.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = data.values.reshape(6000,6000)
    data = np.where(data != 65535, data ,np.nan)
    data = data * 0.0001
    SR_file=open(output_path,'wb')
    data.astype('f4').tofile(SR_file)
    SR_file.close()
    
def SGLI_2_AHI_GEO(filepath,output_path):
    geotiff_da = xr.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = data.values.reshape(6000,6000)
    data = np.where(data != -32768, data ,np.nan)
    data = data * 0.01
    SR_file=open(output_path,'wb')
    data.astype('f4').tofile(SR_file)
    SR_file.close()
        
        
        
def save_raster ( output_name, dataset, driver):
    # Open the reference dataset
    g = ( dataset )
    # Get the Geotransform vector
    geo_transform = g.GetGeoTransform ()
    x_size = g.RasterXSize # Raster xsize
    y_size = g.RasterYSize # Raster ysize
    srs = g.GetProjectionRef () # Projection
    raster_data = g.ReadAsArray()
    driver = gdal.GetDriverByName ( driver )
    dataset_out = driver.Create ( output_name, x_size, y_size, 1, \
            gdal.GDT_Float32 )
    dataset_out.SetGeoTransform ( geo_transform )
    dataset_out.SetProjection ( srs )
    dataset_out.GetRasterBand ( 1 ).WriteArray ( \
            raster_data.astype(np.float32) )
    

def Mosaic_IMAGE(filelist,output_path):
    
    gdal.BuildVRTOptions(VRTNodata = 65535,srcNodata=65535)
    b1 = gdal.BuildVRT("dummy", filelist, VRTNodata=65535, srcNodata=65535) 
    save_raster(output_path,b1,"GTiff")
    
def Mosaic_GEO(filelist,output_path):
    
    gdal.BuildVRTOptions(VRTNodata = -32768,srcNodata=-32768)
    b1 = gdal.BuildVRT("dummy", filelist, VRTNodata=-32768, srcNodata=-32768) 
    save_raster(output_path,b1,"GTiff")

In [8]:
def download_tile(date,save_tile):
    tile_file_name = []
    count = 0
    tile = ['0427','0428','0429','0527','0528','0529','0627','0628','0629']
    for t in tile:
        g = SGLI_L2_Downloader(
            'galiwei ' ,
            'RSRF', 
            '3', 
             date , 
             t, 
            'D' , 
            '1' ,
             save_tile
            )
        if count == 0:
            f = g.login()
        g.download(f)
        fn = g.filename()
        tile_file_name.append(fn) 
    g.logout(f)
    
    return tile_file_name

In [62]:
import os
from osgeo import gdal
import numpy as np
import xarray as xr



YYYY = '2022'
MM = ['01']
#DD = ['07','09','11','13','15',]
DD = ['07']

lat_x=np.linspace(50,20,6000)
lon_y=np.linspace(120,150,6000)

band = ['Rs_VN04',\
        'Solar_azimuth', 'Solar_zenith','Sensor_zenith', 'Sensor_azimuth', \
        'Solar_azimuth_PL', 'Solar_zenith_PL','Sensor_zenith_PL', 'Sensor_azimuth_PL']

for k in range(len(MM)):
    for m in range(len(DD)):
        
        date = YYYY+MM[k]+DD[m]
        print('Start {}'.format(date))
        save_tile = '/media/liwei/Data/GCOM-C_RSRF/tile/'
        save_Mosaic = '/media/liwei/Data/GCOM-C_RSRF/Mosaic/'
        save_reporjection = '/media/liwei/Data/GCOM-C_RSRF/reporjection/'
        tile_file_name = download_tile(date,save_tile)
        
        for b in band:
            print('Start Reporjection {}'.format(b))
            for fn in tile_file_name:
                if b[0:2] == 'Rs':
                    reporjection_Image(save_tile+fn,b,save_reporjection)
                else:
                    reporjection_Geo(save_tile+fn,b,save_reporjection)
                    
            print('Reporjection Finished!')
            print('Start Mosaic')
            
            save_path = '/media/liwei/Data/GCOM-C_RSRF/{}/{}/'.format(b,date)
            
            mkdir(save_path)
            files = os.listdir(save_reporjection)
            files.sort()
            files_tif = []
            for file in files:
                if file[-3:] == 'tif':
                    files_tif.append(save_reporjection + file)
                    
            
            
            if b[0:2] == 'Rs':
                Mosaic_IMAGE(files_tif,save_Mosaic+'{}_{}.tif'.format(b,date))
                print('Mosaic Finished')
                os.system('rm -f {}*'.format(save_reporjection))
                print('{} Reporjection Deleted Finished'.format(b))
    
                SGLI_2_AHI_IMAGE(save_Mosaic+'{}_{}.tif'.format(b,date),save_path+'{}_{}.dat'.format(b,date))
            else:
                Mosaic_GEO(files_tif,save_Mosaic+'{}_{}.tif'.format(b,date))
                print('Mosaic Finished')
                os.system('rm -f {}*'.format(save_reporjection))
                print('{} Reporjection Deleted Finished'.format(b))
                SGLI_2_AHI_GEO(save_Mosaic+'{}_{}.tif'.format(b,date),save_path+'{}_{}.dat'.format(b,date))
            
            
            print('Resample to AHI Finished')
            os.system('rm -f {}*'.format(save_Mosaic))
            print('{} Mosaic Data Deleted Finished'.format(b))
                
        os.system('rm -f {}*'.format(save_tile)) 
        print('{} Tile Data Deleted Finished'.format(date))

            

Start 20220107
Start Reporjection Rs_VN04
Reporjection Finished!
Start Mosaic
Mosaic Finished
Rs_VN04 Reporjection Deleted Finished
Resample to AHI Finished
Rs_VN04 Mosaic Data Deleted Finished
Start Reporjection Solar_azimuth
Reporjection Finished!
Start Mosaic
Mosaic Finished
Solar_azimuth Reporjection Deleted Finished
Resample to AHI Finished
Solar_azimuth Mosaic Data Deleted Finished
Start Reporjection Solar_zenith
Reporjection Finished!
Start Mosaic
Mosaic Finished
Solar_zenith Reporjection Deleted Finished
Resample to AHI Finished
Solar_zenith Mosaic Data Deleted Finished
Start Reporjection Sensor_zenith
Reporjection Finished!
Start Mosaic
Mosaic Finished
Sensor_zenith Reporjection Deleted Finished
Resample to AHI Finished
Sensor_zenith Mosaic Data Deleted Finished
Start Reporjection Sensor_azimuth
Reporjection Finished!
Start Mosaic
Mosaic Finished
Sensor_azimuth Reporjection Deleted Finished
Resample to AHI Finished
Sensor_azimuth Mosaic Data Deleted Finished
Start Reporjection